In [1]:
import time
import datetime
principio = datetime.datetime.now()
principio

datetime.datetime(2024, 1, 5, 12, 42, 26, 115916)

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
df = pd.read_csv('../data/datos sin días.csv')
df.columns = ['ds', 'y']
df['ds']=pd.to_datetime(df['ds'])


In [4]:
def mean_absolute_percentage_error(y_true, y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  if np.any(y_true==0)==True:
    print('Erro: y_true contêm zeros!')
    return np.inf
  else:
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [5]:
exclude_last_n = 31
train = df.iloc[:-exclude_last_n]
test = df.iloc[-exclude_last_n:]

In [6]:
import warnings
warnings.filterwarnings('ignore')

from neuralprophet import NeuralProphet
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

# sp_values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 30, 31, 36, 365, 366]
# funcionperdida = ['MSE', 'MAE', 'Huber']
# sm_values = ["additive", "multiplicative"]
# tscv = TimeSeriesSplit(n_splits=5)  # Puedes ajustar el número de divisiones según tus necesidades

# for h in funcionperdida:
#     for i in sp_values:
#         for j in sm_values:

#             mape_scores = []
    
#             for train_index, test_index in tscv.split(train):
#                 train_cv, test_cv = train.iloc[train_index], train.iloc[test_index]
    
#                 model = NeuralProphet(
#                     growth="linear",
#                     changepoints=None,
#                     n_changepoints=15,
#                     changepoints_range=0.8,
#                     trend_reg=0,
#                     trend_reg_threshold=False,
#                     yearly_seasonality="auto",
#                     weekly_seasonality="auto",
#                     daily_seasonality="auto",
#                     seasonality_mode=j,
#                     seasonality_reg=i,
#                     n_forecasts=1,
#                     n_lags=0,
#                     epochs=50,
#                     loss_func=h,
#                     normalize="auto",
#                     impute_missing=True
#                 )
                
#                 # Ajustar el modelo con el conjunto de entrenamiento actual
#                 metrics = model.fit(train_cv, freq='D')
    
#                 # Hacer predicciones en el conjunto de prueba actual
#                 future_cv = model.make_future_dataframe(train_cv, periods=len(test_cv), n_historic_predictions=len(train_cv))
#                 forecast_cv = model.predict(future_cv)
    
#                 # Calcular el MAPE para el conjunto de prueba actual
#                 y_true_cv = test_cv['y'].values
#                 y_pred_cv = forecast_cv['yhat1'][-len(test_cv):].values
#                 mape_cv = mean_absolute_percentage_error(y_true_cv, y_pred_cv)
    
#                 mape_scores.append(mape_cv)
    
#             avg_mape = np.mean(mape_scores)
#             print(f'Con sr = {i}, lf: {h}, y sm: {j} AVG MAPE: {avg_mape:.2f}%')

Con sr = 12, lf: MSE, y sm: additive AVG MAPE: 114.99%

In [7]:
model = NeuralProphet(
            growth="linear",    # Determine trend types: 'linear', 'discontinuous', 'off'
            changepoints=None,  # list of dates that may include change points (None->automatic)
            n_changepoints=15,
            changepoints_range=0.8,
            trend_reg=0,
            trend_reg_threshold=False,
            yearly_seasonality="auto",
            weekly_seasonality="auto",
            daily_seasonality="auto",
            seasonality_mode="additive",
            seasonality_reg=12,
            n_forecasts=1,
            n_lags=0, # modeificar
            epochs=50, #  50
            loss_func="MSE",
            normalize="auto",   # Type of normalization ('minmax', 'standardize', 'soft', 'off')
            impute_missing=True
        )

INFO - (NP.config.__post_init__) - Note: Fourier-based seasonality regularization is experimental.


In [8]:
metrics = model.fit(train, freq='D')

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.93% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (23) is too small than the required number                     for the learning rate finder (229). The results might not be optimal.


Finding best initial lr:   0%|          | 0/229 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

In [9]:
future = model.make_future_dataframe(train, periods=len(test), n_historic_predictions=len(train))
forecast = model.predict(future)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.93% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.932% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.932% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.data.processing._handle_missing_data) - Dropped 31 rows at the end with NaNs in 'y' column.


Predicting: 23it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


In [10]:
import numpy as np
from sklearn.metrics import mean_absolute_error
y_true = test['y'].values
y_pred = forecast['yhat1'][-31:].values
mae  = mean_absolute_error(y_true, y_pred)
mape = mean_absolute_percentage_error(y_true, y_pred)

print(f'MAE:  {mae:.2f} ventas')
print(f'MAPE: {mape:.2f}%')

MAE:  3704.73 ventas
MAPE: 34.45%


In [11]:
final = datetime.datetime.now()
duracion = final-principio
print(principio)
print(final)
print(duracion)

2024-01-05 12:42:26.115916
2024-01-05 12:42:56.232461
0:00:30.116545
